In [ ]:
import tensorflow as tf
import cv2
from skimage.feature import hog, local_binary_pattern
from skimage import exposure
import numpy as np
from skimage.transform import resize
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

In [ ]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
def extract_hog_features(images):
    hog_features = []
    for image in images:
        fd, _ = hog(image, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=False)
        hog_features.append(fd)
    return np.array(hog_features)

In [ ]:
def extract_lbp_features(images):
    lbp_features = []
    for image in images:
        lbp = local_binary_pattern(image, 8, 1, method='uniform')
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-7)
        lbp_features.append(hist)
    return np.array(lbp_features)

In [ ]:
# Apply feature extraction methods
x_train_hog = extract_hog_features(x_train)
x_train_lbp = extract_lbp_features(x_train)

<ipython-input-3-772185695394>:4: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd, _ = hog(image, orientations=8, pixels_per_cell=(16, 16),


In [ ]:
# Print shapes of extracted features
print("HoG features shape:", x_train_hog.shape)
print("LBP features shape:", x_train_lbp.shape)

HoG features shape: (60000, 8)
LBP features shape: (60000, 9)


In [ ]:
# Concatenate extracted features
x_train_features = np.concatenate((x_train_hog, x_train_lbp), axis=1)

# Split data into training and testing sets
x_train_features, x_val_features, y_train, y_val = train_test_split(x_train_features, y_train, test_size=0.2, random_state=42)

In [ ]:
# Initialize models
svm_model = SVC()
rf_model = RandomForestClassifier()
knn_model = KNeighborsClassifier()
dt_model = DecisionTreeClassifier()
nb_model = GaussianNB()

In [ ]:
# Train models
svm_model.fit(x_train_features, y_train)
rf_model.fit(x_train_features, y_train)
knn_model.fit(x_train_features, y_train)
dt_model.fit(x_train_features, y_train)
nb_model.fit(x_train_features, y_train)

GaussianNB()

In [ ]:
# Predict on validation set
svm_preds = svm_model.predict(x_val_features)
rf_preds = rf_model.predict(x_val_features)
knn_preds = knn_model.predict(x_val_features)
dt_preds = dt_model.predict(x_val_features)
nb_preds = nb_model.predict(x_val_features)

In [ ]:
# Calculate accuracy
svm_accuracy = accuracy_score(y_val, svm_preds)
rf_accuracy = accuracy_score(y_val, rf_preds)
knn_accuracy = accuracy_score(y_val, knn_preds)
dt_accuracy = accuracy_score(y_val, dt_preds)
nb_accuracy = accuracy_score(y_val, nb_preds)

In [ ]:
# Print accuracy
print("Support Vector Machine Accuracy:", svm_accuracy)
print("Random Forest Accuracy:", rf_accuracy)
print("K-Nearest Neighbors Accuracy:", knn_accuracy)
print("Decision Tree Accuracy:", dt_accuracy)
print("Gaussian Naive Bayes Accuracy:", nb_accuracy)

Support Vector Machine Accuracy: 0.5656666666666667
Random Forest Accuracy: 0.6225833333333334
K-Nearest Neighbors Accuracy: 0.4796666666666667
Decision Tree Accuracy: 0.48383333333333334
Gaussian Naive Bayes Accuracy: 0.5165833333333333


In [ ]:
# Calculate evaluation metrics
svm_report = classification_report(y_val, svm_preds)
rf_report = classification_report(y_val, rf_preds)
knn_report = classification_report(y_val, knn_preds)
dt_report = classification_report(y_val, dt_preds)
nb_report = classification_report(y_val, nb_preds)

In [ ]:
# Print evaluation metrics
print("Support Vector Machine Evaluation Report:")
print(svm_report)
print("\nRandom Forest Evaluation Report:")
print(rf_report)
print("\nK-Nearest Neighbors Evaluation Report:")
print(knn_report)
print("\nDecision Tree Evaluation Report:")
print(dt_report)
print("\nGaussian Naive Bayes Evaluation Report:")
print(nb_report)

Support Vector Machine Evaluation Report:
              precision    recall  f1-score   support

           0       0.63      0.65      0.64      1175
           1       0.94      0.93      0.94      1322
           2       0.46      0.38      0.41      1174
           3       0.49      0.46      0.48      1219
           4       0.49      0.43      0.46      1176
           5       0.37      0.24      0.29      1104
           6       0.62      0.66      0.64      1177
           7       0.65      0.62      0.64      1299
           8       0.55      0.69      0.61      1160
           9       0.38      0.54      0.44      1194

    accuracy                           0.57     12000
   macro avg       0.56      0.56      0.55     12000
weighted avg       0.57      0.57      0.56     12000


Random Forest Evaluation Report:
              precision    recall  f1-score   support

           0       0.70      0.73      0.71      1175
           1       0.96      0.96      0.96      1322
  

In [ ]:
# Perform cross-validation
svm_scores = cross_val_score(svm_model, x_train_features, y_train, cv=5)
rf_scores = cross_val_score(rf_model, x_train_features, y_train, cv=5)
knn_scores = cross_val_score(knn_model, x_train_features, y_train, cv=5)
dt_scores = cross_val_score(dt_model, x_train_features, y_train, cv=5)
nb_scores = cross_val_score(nb_model, x_train_features, y_train, cv=5)

In [ ]:
# Print cross-validation scores
print("Support Vector Machine Cross-Validation Scores:", svm_scores)
print("Random Forest Cross-Validation Scores:", rf_scores)
print("K-Nearest Neighbors Cross-Validation Scores:", knn_scores)
print("Decision Tree Cross-Validation Scores:", dt_scores)
print("Gaussian Naive Bayes Cross-Validation Scores:", nb_scores)

Support Vector Machine Cross-Validation Scores: [0.5696875  0.56395833 0.56958333 0.5675     0.56489583]
Random Forest Cross-Validation Scores: [0.63145833 0.62697917 0.63125    0.62989583 0.618125  ]
K-Nearest Neighbors Cross-Validation Scores: [0.489375   0.48125    0.48895833 0.47666667 0.48072917]
Decision Tree Cross-Validation Scores: [0.48125    0.49020833 0.48739583 0.48177083 0.4815625 ]
Gaussian Naive Bayes Cross-Validation Scores: [0.52979167 0.51833333 0.52677083 0.52041667 0.5171875 ]


In [ ]:
# Calculate mean and standard deviation of scores
print("Mean Accuracy (SVM):", svm_scores.mean())
print("Standard Deviation of Accuracy (SVM):", svm_scores.std())
print("\nMean Accuracy (Random Forest):", rf_scores.mean())
print("Standard Deviation of Accuracy (Random Forest):", rf_scores.std())
print("\nMean Accuracy (K-Nearest Neighbors):", knn_scores.mean())
print("Standard Deviation of Accuracy (K-Nearest Neighbors):", knn_scores.std())
print("\nMean Accuracy (Decision Tree):", dt_scores.mean())
print("Standard Deviation of Accuracy (Decision Tree):", dt_scores.std())
print("\nMean Accuracy (Gaussian Naive Bayes):", nb_scores.mean())
print("Standard Deviation of Accuracy (Gaussian Naive Bayes):", nb_scores.std())

Mean Accuracy (SVM): 0.5671250000000001
Standard Deviation of Accuracy (SVM): 0.002355733254565685

Mean Accuracy (Random Forest): 0.6275416666666667
Standard Deviation of Accuracy (Random Forest): 0.004972406498757793

Mean Accuracy (K-Nearest Neighbors): 0.48339583333333336
Standard Deviation of Accuracy (K-Nearest Neighbors): 0.0049737156354401875

Mean Accuracy (Decision Tree): 0.48443749999999997
Standard Deviation of Accuracy (Decision Tree): 0.003676714452696414

Mean Accuracy (Gaussian Naive Bayes): 0.5225
Standard Deviation of Accuracy (Gaussian Naive Bayes): 0.0049261032323283166
